# Objective

This notebook covers how to get started with the **Objective** vector store.

Objective is an engine for building modern, AI-native search.

Key features include:

* Semantic, automatically understanding natural language queries, synonyms, typos, and multiple languages.
* Hybrid, capable of exact matching and approximate matching in one API without requiring you to develop lexical / keyword matching as well as vector-based approximate nearest neighbors (ANN) engines.
* Deep, surfacing relevant Highlights from different media by going inside the content and pulling out relevant bits.

## Setup

To use Objective be sure to install the latest `langchain-community` with `pip install -qU langchain-community`.

### Credentials

Next you'll need to [sign up](https://app.objective.inc/) and copy your [API key](https://app.objective.inc/dashboard).

The easiest step to configure your key is to store it in the `OBJECTIVE_KEY` environment variable. Otherwise you may use it directly below

In [ ]:
import os

objective_key = os.getenv("OBJECTIVE_KEY")

## Initialization

In [ ]:
from langchain_community.vectorstores import Objective

vector_store = Objective(objective_key)

## Manage vector store

### Add items to vector store

In [ ]:
from langchain_core.documents import Document

document_1 = Document(page_content="foo", metadata={"source": "https://example.com"})

document_2 = Document(page_content="bar", metadata={"source": "https://example.com"})

document_3 = Document(page_content="baz", metadata={"source": "https://example.com"})

documents = [document_1, document_2, document_3]

vector_store.add_documents(documents=documents, ids=["1", "2", "3"])

### Update items in vector store

In [ ]:
updated_document = Document(
    id="1", page_content="qux", metadata={"source": "https://another-example.com"}
)

vector_store.upsert([updated_document])

### Delete items from vector store

In [ ]:
vector_store.delete(ids=["3"])

## Create a searchable index

In [ ]:
index_id = vector_store.create_index()
print(f"Created index ID: {index_id}")

In [ ]:
status = vector_store.index_status(index_id)
# status returns an object with UPLOADED, READY, ERROR, and PROCESSING document counts
if status["UPLOADED"] != status["READY"]:
    print(f"Not all documents processed yet, please retry: {status}")
else:
    print("Ready to proceed")

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent.

### Query directly

In [ ]:
results = vector_store.search(query="thud", search_type="similarity", index_id=index_id)
for doc in results:
    print(doc.page_content)

### Query by turning into retriever

You can also transform the vector store into a retriever for easier usage in your chains.

In [ ]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("thud")

## Usage for retrieval-augmented generation

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [Tutorials: working with external knowledge](https://python.langchain.com/v0.2/docs/tutorials/#working-with-external-knowledge)
- [How-to: Question and answer with RAG](https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](https://python.langchain.com/v0.2/docs/concepts/#retrieval)

## Chain usage

The code below shows how to use the vector store as a retriever in a simple RAG chain:

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("thud")

## API reference